In [79]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [216]:
# %load_ext autoreload
%autoreload 2
from pyspark.sql import SQLContext
import boto3
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from io import StringIO
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sk_model import ReviewClassifier, metrics, vectorizer
import util

In [82]:
f_path = "s3://chizzy/yelp_data/"
df_bus = pd.read_json(f_path+"business.json", lines=True)
df_user = pd.read_json(f_path+"user.json", lines=True)
df_rev = pd.read_json(f_path+"review.json", lines=True)

In [83]:
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [84]:
df_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08


In [85]:
df_rev.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [86]:
#Rename colume names 
df_bus.rename(columns={"name":"bus_name","review_count":"bus_rev_count"}, inplace=True)
df_user.rename(columns={"name":"user_name", "review_count":"user_rev_count"}, inplace=True)
df_rev.rename(columns={"cool":"cool_rev", "funny":"funny_rev", "stars":"stars_rev", "useful":"useful_rev"}, inplace=True)

In [87]:
print("Number of rows for business table: {}".format(len(df_bus)))
print("Number of rows for user table: {}".format(len(df_user)))
print("Number of rows for review table: {}".format(len(df_rev)))

Number of rows for business table: 174567
Number of rows for user table: 1326101
Number of rows for review table: 5261669


In [88]:
us_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS", \
            "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY", \
            "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

#Extracting reviews for only US restaurants
df_bus_us = df_bus[df_bus.state.isin (us_states)]

In [89]:
print("Number of rows for business table for US states: {}".format(len(df_bus_us)))

Number of rows for business table for US states: 128302


In [90]:
df_rev_bus = pd.merge(df_rev, df_bus, how='inner', on=['business_id', 'business_id'])

In [91]:
df_rev_bus.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,hours,is_open,latitude,longitude,bus_name,neighborhood,postal_code,bus_rev_count,stars,state
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC


In [92]:
df_rev_bus_user = pd.merge(df_rev_bus, df_user, how='left', on=['user_id', 'user_id'])

In [93]:
print("Number of rows for business table for concated business, user and review: {}".format(len(df_rev_bus_user)))

Number of rows for business table for concated business, user and review: 5261669


In [94]:
df_rev_bus_user.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,compliment_writer,cool,elite,fans,friends,funny,user_name,user_rev_count,useful,yelping_since
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,0,0,[],0,[4Zpn3ObC2zexkIhor4LJIQ],0,Tim,6,0,2015-12-22
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,3,22,"[2012, 2014, 2016, 2017, 2015, 2013]",8,"[QSpRpbemKx5U4-NtUumUbw, G-6X-llgA_qAxGxocykHz...",7,Julia,359,79,2009-08-20
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,8,0,"[2015, 2017, 2016]",30,"[UX0Lh2n7gOiH5PVsCo1psA, GM7NRYBXxl3ELf8y4Loq0...",14,Anh,177,39,2015-02-15
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,0,0,[],1,"[_V28Y9fkcQgo3-F24DDxTA, ZCIvJM3fuFYpUFIG4OFzj...",0,Tuan,26,1,2012-02-03
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,1,6,[2017],1,"[PhJhg24zecO2USi_blq6eg, uOQTLqNUtiuNWxaTma02E...",3,Kenneth,32,10,2015-08-15


In [95]:
df_rev_bus_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5261669 entries, 0 to 5261668
Data columns (total 44 columns):
business_id           object
cool_rev              int64
date                  datetime64[ns]
funny_rev             int64
review_id             object
stars_rev             int64
text                  object
useful_rev            int64
user_id               object
address               object
attributes            object
categories            object
city                  object
hours                 object
is_open               int64
latitude              float64
longitude             float64
bus_name              object
neighborhood          object
postal_code           object
bus_rev_count         int64
stars                 float64
state                 object
average_stars         float64
compliment_cool       int64
compliment_cute       int64
compliment_funny      int64
compliment_hot        int64
compliment_list       int64
compliment_more       int64
compliment_note  

In [100]:
# with open("s3://chizzy/yelp_data/df_rev_bus_user.pickle", "wb") as f:
#     pickle.dump(df_rev_bus_user, f)

In [101]:
# with open("s3://chizzy/yelp_data/df_rev_bus_user.pickle", "rb") as f:
#     df = pickle.load(f)

In [102]:
df = df_rev_bus_user[["text", "stars_rev"]][df_rev_bus_user["state"]=="WI"]
df.head()

,text,stars_rev
13480,We got a permit to have wedding photos taken h...,4
13481,Great place any time of the year. The family a...,4
13482,"Gorgeous, peaceful, relaxing, but still being ...",5
13483,"Free and much larger than I expected, this bot...",5
13484,Fantastic - so many things to see in the conse...,5


In [103]:
len(df)

109751

In [104]:

corpus = df["text"].values

In [105]:
print("Number of words reviews in corpus: {}".format(len(corpus)))

Number of words reviews in corpus: 109751


In [122]:
corpus[1]

'Great place any time of the year. The family and I went last weekend just to see what was going on.  Even though there were no special events, we were welcomed by friendly staff and walked the conservancy.  There was a good number of people walking the path through the recreated rain forest, though not an overwhelming amount.  We enjoyed exploring the works of Maria Sibylla Merian while taking in the sights and smells.\n\nWe have been to this local treasure in the past and will continue to return.  Any time of the year is the right time for Olbrich.'

In [107]:
print("Number of words befor cleaning, and removing stopwords: {}".format(len(corpus[3].split())))

Number of words befor cleaning, and removing stopwords: 46


In [108]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [199]:
sw = set(stopwords.words("english"))
tokenizer = RegexpTokenizer("[\w']+")
st = PorterStemmer()
lemma = WordNetLemmatizer()

In [200]:
cleaned = util.clean_stem(corpus, tokenizer, lemma, sw)

In [201]:
cleaned[1]

'great place time year family went last weekend see going even though special event welcomed friendly staff walked conservancy good number people walking path recreated rain forest though overwhelming amount enjoyed exploring work maria sibylla merian taking sight smell local treasure past continue return time year right time olbrich'

In [202]:
print("Number of words before cleaning, and removing stopwords: {}".format(len(corpus[3].split())))
print("Number of words after cleaning, and removing stopwords: {}".format(len(cleaned[3].split())))

Number of words before cleaning, and removing stopwords: 46
Number of words after cleaning, and removing stopwords: 25


In [203]:
X, terms = vectorizer(cleaned, 
                       max_df=0.95, 
                       min_df=2,
                       stop_words='english')

In [204]:
terms

['aa',
 'aaa',
 'aaaand',
 'aaaannnd',
 'aaah',
 'aaahhh',
 'aaand',
 'aaargh',
 'aah',
 'aaloo',
 'aaron',
 'aarp',
 'ab',
 'aba',
 'aback',
 'abacus',
 'abalone',
 'abandon',
 'abandoned',
 'abandoning',
 'abarrotes',
 'abbey',
 'abbreviated',
 'abbreviation',
 'abby',
 'abc',
 'abd',
 'abdomen',
 'abdominal',
 'abductor',
 'abe',
 'aber',
 'abercrombie',
 'aberration',
 'abf',
 'abhor',
 'abhorrent',
 'abhorrently',
 'abide',
 'abiding',
 'abigail',
 'ability',
 'abit',
 'abita',
 'abject',
 'ablaze',
 'able',
 'abled',
 'ables',
 'ablt',
 'abnormal',
 'abnormality',
 'abnormally',
 'aboard',
 'abode',
 'abolutely',
 'abominable',
 'abomination',
 'abort',
 'aborted',
 'abortion',
 'aboslutely',
 'abou',
 'abound',
 'abounded',
 'abounding',
 'abounds',
 'abra',
 'abraham',
 'abrams',
 'abrasion',
 'abrasive',
 'abreast',
 'abroad',
 'abrupt',
 'abruptly',
 'abruzzo',
 'absence',
 'absent',
 'absentee',
 'absinthe',
 'absoluetly',
 'absolut',
 'absolute',
 'absolutely',
 'absolutley

In [205]:
# tfidf_vectorizer= TfidfVectorizer(max_df=0.95, min_df=2,
#                                 stop_words='english')

# tfidf = tfidf_vectorizer.fit_transform(cleaned)

In [206]:
X.shape

(109751, 34051)

In [207]:
y = df["stars_rev"].values

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### LogisticRegression Linear model

In [231]:
model = ReviewClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [233]:
y_test

array([5, 4, 2, ..., 3, 5, 4])

In [232]:
confusion_matrix(y_test, y_pred)

array([[2205,  364,  109,  114,  237],
       [ 679,  723,  521,  380,  263],
       [ 193,  349,  932, 1486,  566],
       [  73,   62,  352, 3840, 3138],
       [  57,   18,   68, 1732, 8977]])

In [222]:
matrix, recall, precision, accuracy = model.metrics_eval(X_test, y_test)

In [223]:
#confusion matrix
matrix

array([[12817,  7916],
       [    0,  3860]])

In [224]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 100.0%
Precision: 61.82%
Accuracy: 67.81%


In [228]:
# terms = tfidf_vectorizer.get_feature_names()
pos_terms, neg_terms = model.feature_importance(terms,40)

In [229]:
pos_terms

[('amazing', 7.083614093183224),
 ('best', 6.504054461116863),
 ('highly', 5.001365932301159),
 ('favorite', 4.980809375535258),
 ('perfect', 4.335904457190401),
 ('fantastic', 4.264483640859959),
 ('awesome', 4.208074782371054),
 ('incredible', 4.13574290070926),
 ('excellent', 3.996342392206019),
 ('thank', 3.825703236808566),
 ('love', 3.7921354711909796),
 ('great', 3.6946831814105456),
 ('delicious', 3.6784978239215262),
 ('wonderful', 3.581756318757367),
 ('professional', 3.5295800419125487),
 ('outstanding', 3.4328927427332645),
 ('madison', 3.3533280375046544),
 ('phenomenal', 3.250557310566473),
 ('knowledgeable', 3.2330479878494707),
 ('notch', 3.14085890944572),
 ('wow', 3.109817282812256),
 ('gem', 3.0937975525221892),
 ('dr', 3.0462343649520798),
 ('perfection', 2.9919554462075717),
 ('impeccable', 2.9772956865379503),
 ('dream', 2.9233808094611202),
 ('happier', 2.877373322299774),
 ('die', 2.833518533676713),
 ('perfectly', 2.8174343887674107),
 ('thanks', 2.795264182552

In [230]:
neg_terms

[('worst', -6.988844142128644),
 ('ok', -6.39827132736458),
 ('rude', -6.2139552931838296),
 ('mediocre', -5.524089369726015),
 ('okay', -5.493919896033115),
 ('terrible', -5.2826082797229414),
 ('horrible', -5.272811912348712),
 ('disappointing', -5.241549954154296),
 ('bland', -5.226577172173807),
 ('decent', -5.07591651856392),
 ('overpriced', -5.010894224846887),
 ('awful', -4.747483957572529),
 ('average', -4.400254794158546),
 ('pretty', -4.218476288424618),
 ('unfortunately', -4.1954304454645115),
 ('slow', -4.185083832249633),
 ('overall', -4.042990538160044),
 ('meh', -4.024155025669153),
 ('guess', -3.8928846563884805),
 ('poor', -3.8446078818801546),
 ('lack', -3.725275209859661),
 ('worse', -3.6289615473913948),
 ('lacked', -3.6220511630475323),
 ('told', -3.60938478565554),
 ('reason', -3.571176764443839),
 ('alright', -3.4644434824804637),
 ('said', -3.307596233949542),
 ('star', -3.1433283388288005),
 ('barely', -3.1193340021475264),
 ('solid', -3.112581257750921),
 ('la